In [11]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [12]:
# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('Important Codes/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']

In [13]:
# Create an ImageDataGenerator object
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Set the validation split here
)

In [14]:
# Training generator
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # or 'categorical' if more than two classes
    subset='training'  # Specify that this is the training subset
)

# Validation generator
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # or 'categorical' if more than two classes
    subset='validation'  # Specify that this is the validation subset
)

Found 176 images belonging to 2 classes.
Found 43 images belonging to 2 classes.


In [15]:
# Define the model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Adding dropout; 50% of the layer's activations will be set to zero
    Dense(1, activation='sigmoid')  # For binary classification
])

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [17]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,105 (56.61 MB)

 Trainable params: 14,839,105 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/50


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.4908 - loss: 1.3025 - val_accuracy: 0.4375 - val_loss: 0.7004
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3438 - loss: 0.7057 - val_accuracy: 0.5455 - val_loss: 0.6907
Epoch 3/50


/opt/anaconda3/envs/Thesis/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 473ms/step - accuracy: 0.5286 - loss: 0.6941
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 316ms/step - accuracy: 0.6875 - loss: 0.6707 - val_accuracy: 0.6250 - val_loss: 0.6748
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 396ms/step - accuracy: 0.5612 - loss: 0.6931 - val_accuracy: 0.3636 - val_loss: 0.7556
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5625 - loss: 0.6975  
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 708ms/step - accuracy: 0.4956 - loss: 0.7101 - val_accuracy: 0.1875 - val_loss: 0.7166
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5625 - loss: 0.6797 - val_accuracy: 0.0909 - val_loss: 0.7059
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 401ms/step - accuracy: 0.5296 - loss: 0.6891
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step - accuracy: 0.7188 - loss: 0.6672 - val_accuracy: 0.6250 - val_loss: 0.6944
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 473ms/step - accuracy: 0.5418 - loss: 0.6887 - val_accuracy: 0.3636 - val_loss: 0.8007
Ep

2024-10-22 21:56:59.757606: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 0.7500 - loss: 0.6508 - val_accuracy: 0.5000 - val_loss: 0.7033
Epoch 29/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 416ms/step - accuracy: 0.6401 - loss: 0.6386 - val_accuracy: 0.3636 - val_loss: 0.8291
Epoch 30/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7188 - loss: 0.5821  
Epoch 31/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 738ms/step - accuracy: 0.7538 - loss: 0.6235 - val_accuracy: 0.4688 - val_loss: 0.8615
Epoch 32/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6875 - loss: 0.5621 - val_accuracy: 0.7273 - val_loss: 0.6689
Epoch 33/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 409ms/step - accuracy: 0.7065 - loss: 0.5727
Epoch 34/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 254ms/step - accuracy: 0.8750 - loss: 0.4710 - val_accuracy: 0.5000 - val_loss: 0.8799
Epoch 35/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 417ms/step - accuracy: 0.6369 - loss: 0.6142 - val_accuracy: 0.2727 - val_loss: 1.0589
Epoch 36/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7812 - loss: 0.522

In [19]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step - accuracy: 0.4922 - loss: 0.9432
Test Accuracy: 48.84%
